### [스티커모으기(2)](https://school.programmers.co.kr/learn/courses/30/lessons/12971)

In [10]:
def solution(sticker):
    answer = 0
    n = len(sticker)
    if n <= 3:
        return max(sticker)
        
    def backtrack(idx, score, last):
        nonlocal answer
        
        if idx >= last :
            answer = max(score + sticker[idx] if idx == last else score, answer)
            return
            
        # 현재 idx를 선택하는 경우
        backtrack(idx+2, score+sticker[idx], last)
        # 선택하지 않는 경우
        backtrack(idx+1, score, last)
        
    
    backtrack(2, sticker[0], n-2) # 처음 걸 선택하는 경우(마지막 것도 선택 못함)
    backtrack(1, 0, n-1)          # 처음 걸 선택하지 않는 경우(마지막 것 선택 가능)
    
    return answer

In [11]:
sticker = [14, 6, 5, 11, 3, 9, 2, 10]
print(solution(sticker))

36


- 속도가 너무 느림 -> dp로 풀어야할듯?

In [ ]:
def solution(sticker):
    n = len(sticker)
    if n <= 3:
        return max(sticker)

    def linear_max(v):
        m = len(v)
        dp = [0]*m
        dp[0] = v[0]
        dp[1] = max(v[0], v[1])
        for i in range(2, m):
            dp[i] = max(dp[i-1], dp[i-2] + v[i])
        return dp[-1]

    # 첫장 포함 → 마지막 미포함
    caseA = linear_max(sticker[:-1])
    # 첫장 미포함 → 마지막 포함 가능
    caseB = linear_max(sticker[1:])
    return max(caseA, caseB)


### [순위검색](https://school.programmers.co.kr/learn/courses/30/lessons/72412)
- 다시 풀기

In [ ]:
from collections import defaultdict
from itertools import combinations

def solution(info, query):
    encoded = defaultdict(list)
    
    # 지원자 정보를 기준으로 모든 패스 가능한 경우의 수 적용
    for string in info:
        target = string.split(' ')
        score = int(target[-1])
        origin = target[:-1]
        
        encoded["".join(origin)].append(score)
        for i in range(1, 4+1):
            for combo in combinations(range(4), i):
                temp = "".join(['-' if e in combo else x for e, x in enumerate(origin)])
                encoded[temp].append(score)
    
    # query 파싱해서 key 찾고, 해당 key의 value(socre) 중 기준 이상인 값을 찾기
    answer = []    
    for q in query:
        required = q.replace('and', '').split()
        k = ''.join(required[:-1])
        v = int(required[-1])
        
        answer.append(len([x for x in encoded[k] if x >= v]))
        
    return answer

- 효율성 기준을 달성못해, 이를 해결하기 위해 이분탐색 도입

In [ ]:
from collections import defaultdict
from itertools import combinations

def solution(info, query):
    encoded = defaultdict(list)
    
    # 지원자 정보를 기준으로 모든 패스 가능한 경우의 수 적용
    for string in info:
        target = string.split(' ')
        score = int(target[-1])
        origin = target[:-1]
        
        encoded["".join(origin)].append(score)
        for i in range(1, 4+1):
            for combo in combinations(range(4), i):
                temp = "".join(['-' if e in combo else x for e, x in enumerate(origin)])
                encoded[temp].append(score)
    
    # 이분탐색 위한 정렬
    for key in encoded.keys():
        encoded[key].sort()
    
    # query 파싱해서 key 찾고, 해당 key의 value(socre) 중 기준 이상인 값을 이분탐색으로 찾기
    answer = []    
    for q in query:
        required = q.replace('and', '').split()
        k = ''.join(required[:-1])
        v = int(required[-1])
        
        scores = encoded[k]
        if not scores:
            answer.append(0)
            continue
            
        # 이분 탐색
        left, right = 0, len(scores)
        while left < right:
            mid = (left + right) // 2
            if scores[mid] >= v: # 중간 지점이 기준보다 큼 -> 왼쪽으로 이동
                right = mid
            else:                # 중간 지점이 기준보다 작거나 같음 -> 오른쪽으로 이동
                left = mid + 1 
        answer.append(len(scores) - left)
        
    return answer